In [1]:
from rating_tools import get_tournament_results_by_id, get_tournaments
from rating_tools import get_all_players, get_player_info, api_call

In [2]:
from collections import Counter
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
import datetime
from dateutil.parser import parse as date_parse
import math
import json

In [279]:
def rolling_window(a, window):
    a = np.append(a, np.zeros(window - 1))
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)


def calculate_bonus_predictions(tournament_ratings, c=1):
    """
    produces array of bonuses based on the array of ratings of participants
    """
    tournament_ratings[::-1].sort()
    raw_preds = np.round(rolling_window(tournament_ratings, 15).dot(2.**np.arange(0, -15, -1)) * c)
    samesies = tournament_ratings[:-1] == tournament_ratings[1:]
    for ind in np.nonzero(samesies)[0]:
        raw_preds[ind + 1] = raw_preds[ind]
    return raw_preds


def get_recaps(tournament_id):
    url = f'https://rating.chgk.info/api/tournaments/{tournament_id}/recaps.json'
    return {x['idteam']: x['recaps'] for x in api_call(url)}


def get_tournaments_for_release(release_date: datetime.datetime):
    result = []
    tournaments_fp = get_tournaments()
    n_tournaments = int(tournaments_fp["total_items"])
    tournaments = tournaments_fp["items"]
    for i in range(1, math.ceil(n_tournaments / 1000) + 1):
        tournaments.extend(get_tournaments(i)["items"])
    for t in tournaments:
        try:
            t_end = date_parse(t["date_end"])
        except ValueError:
            continue
        if t_end < release_date and t_end >= release_date - datetime.timedelta(days=7) and t['type_name'] in {'Обычный', 'Синхрон'}:
            result.append(t)
    return result


def calc_rg(tournament_results, team_release, player_release):
    """
    mock function for now
    returns np.array of rgs
    """
    return np.array([int(t['tech_rating_rg']) for t in tournament_results])


def calc_c(ratings):
    ratings[::-1].sort()
    return 2300 / ratings[:15].dot(2.**np.arange(0, -15, -1))


def calc_score_real(predicted_scores, positions):
    positions = positions - 1
    pos_counts = pd.Series(positions).value_counts().reset_index()
    pos_counts.columns = ['pos', 'n_teams']
    pos_counts['bonus'] = pos_counts.apply(
        lambda x: np.mean(predicted_scores[int(x.pos - (x.n_teams - 1) / 2) : int(x.pos + (x.n_teams - 1) / 2) + 1]), axis=1)
    return np.round(pos_counts.set_index('pos').loc[positions, 'bonus'].values)


def calc_bonus(tournament_df, coeff=0.5):
    """
    tournament_df should have columns: score_real, score_pred, n_leg
    6 ms for bb, kinda long
    """
    d_one = tournament_df.score_real - tournament_df.score_pred
    d_one[d_one < 0] *= 0.5
    d_two = 300 * np.exp((tournament_df.score_real - 2300) / 350)
    d = coeff * (d_one + d_two)
    d[(d > 0) & (tournament_df.n_legs > 2)] *= tournament_df.n_legs[(d > 0) & (tournament_df.n_legs > 2)] 
    return d.astype('int')


#------------------**************------------------------

def calculate_changes(team_release, player_release, tournament_results):
    """
    returns Counter({team_id: change}) based on the tournament results
    """
    rgs = calc_rg(tournament_results, team_release, player_release)
    bonus_predictions = calculate_bonus_predictions(rgs)
    
    return Counter()


def calculate_rating_release(team_release: Counter, player_release, tournaments):
    total_changes = Counter()
    for tournament in tournaments:
        total_changes += calculate_changes(team_release, player_release, tournament) 
    return team_release + total_changes

In [7]:
tournaments_for_release = get_tournaments_for_release(datetime.datetime(2019, 12, 5))

tournaments_results_for_release = {t['idtournament']: get_tournament_results_by_id(t['idtournament']) 
                                   for t in tournaments_for_release}

# with open('tournament_results.json', 'w') as f:
#     json.dump(tournaments_results_for_release, f)

In [265]:
def calc_score_real(predicted_scores, positions):
    positions = positions - 1
    pos_counts = pd.Series(positions).value_counts().reset_index()
    pos_counts.columns = ['pos', 'n_teams']
    pos_counts['bonus'] = pos_counts.apply(
        lambda x: np.mean(predicted_scores[int(x.pos - (x.n_teams - 1) / 2) : int(x.pos + (x.n_teams - 1) / 2) + 1]), axis=1)
    return np.round(pos_counts.set_index('pos').loc[positions, 'bonus'].values)

In [227]:
release = pd.read_csv('release-2019-11-28.csv')
release['Место'] = release['Место'].map(lambda x: x.replace(',', '.')).astype(float)
c = calc_c(release['Рейтинг'][release['Место'] == release['Место'].astype(int)].values)
print(c)

0.11942999030081372


In [282]:
# %%timeit
t_recaps = get_recaps('5752')

In [283]:
t_recaps

{'856': [{'idplayer': '31',
   'is_captain': '0',
   'is_base': '1',
   'is_foreign': '0'},
  {'idplayer': '47526', 'is_captain': '0', 'is_base': '1', 'is_foreign': '0'},
  {'idplayer': '59717', 'is_captain': '0', 'is_base': '1', 'is_foreign': '0'},
  {'idplayer': '59718', 'is_captain': '0', 'is_base': '1', 'is_foreign': '0'},
  {'idplayer': '216195',
   'is_captain': '0',
   'is_base': '0',
   'is_foreign': '0'}],
 '28354': [{'idplayer': '35',
   'is_captain': '0',
   'is_base': '0',
   'is_foreign': '1'},
  {'idplayer': '119', 'is_captain': '0', 'is_base': '1', 'is_foreign': '0'},
  {'idplayer': '409', 'is_captain': '0', 'is_base': '1', 'is_foreign': '0'},
  {'idplayer': '40822', 'is_captain': '0', 'is_base': '1', 'is_foreign': '0'},
  {'idplayer': '54854', 'is_captain': '0', 'is_base': '0', 'is_foreign': '1'}],
 '5858': [{'idplayer': '82',
   'is_captain': '0',
   'is_base': '1',
   'is_foreign': '0'},
  {'idplayer': '4812', 'is_captain': '1', 'is_base': '1', 'is_foreign': '0'},
  {

In [272]:
tourn_pd = pd.DataFrame(tournaments_results_for_release['5752'], dtype='float').set_index('idteam')
tourn_pd = tourn_pd.loc[:, ['position', 'tech_rating_rg']]
tourn_pd.columns = ['position', 'rg']
tourn_pd['n_legs'] = 0

In [273]:
tourn_pd.sort_values(by='tech_rating_rg', ascending=False, inplace=True)
tourn_pd['score_pred'] = calculate_bonus_predictions(tourn_pd.tech_rating_rg.values, c=c)
tourn_pd['score_real'] = calc_score_real(tourn_pd.score_pred.values, tourn_pd.position.values)
tourn_pd['bonus'] = calc_bonus(tourn_pd)

In [274]:
tourn_pd

,position,tech_rating_rg,n_legs,score_pred,score_real,bonus
idteam,,,,,,
74651.0,12.5,8512.0,0,1985.0,1726.0,-35
41366.0,21.0,8241.0,0,1938.0,1618.0,-58
74621.0,12.5,8035.0,0,1907.0,1726.0,-16
53714.0,58.0,7979.0,0,1895.0,1454.0,-96
74622.0,12.5,7979.0,0,1895.0,1726.0,-13
...,...,...,...,...,...,...
75923.0,908.0,0.0,0,0.0,55.0,27
75855.0,908.0,0.0,0,0.0,55.0,27
66373.0,838.0,0.0,0,0.0,189.0,94


In [11]:
%%timeit
calculate_bonus_predictions(calc_rg(tournaments_results_for_release['5752'], 0, 0), c=0.1194)

333 µs ± 18 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [63]:
calculate_rating_release(Counter(), Counter(), [])

Counter()